<a href="https://colab.research.google.com/github/lin2xiao1/delicious_Cn_restaurant_review_analysis/blob/main/%EC%8A%A4%ED%81%AC%EB%9E%98%ED%95%91_%EC%B9%B4%EC%B9%B4%EC%98%A4%EB%A7%B5_%EB%A6%AC%EB%B7%B0_%EC%A0%95%ED%98%9C%EC%97%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## 참조 샘플 코드 예시 -> Colab에서 작동 가능하도록 변경
## https://github.com/wlgh325/python_crawling

!pip install selenium

import sys

# install chromium, its driver, and selenium
if 'google.colab' in sys.modules:
    !apt-get update
    !apt install chromium-chromedriver
    !cp /usr/lib/chromium-browser/chromedriver /usr/bin
    !pip install selenium

# set options to be headless, ..
import os
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

##############################################################  ############
##################### variable related selenium ##########################
##########################################################################
#chromedriver_path = "chromedriver"
#driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기

     |████████████████████████████████| 911kB 7.2MB/s 
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchp

In [ ]:
result_list = []

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
#options.add_argument('headless')
#options.add_argument('lang=ko_KR')

driver = webdriver.Chrome('chromedriver',options=options) 

In [ ]:
def search(place):
    global driver

    search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')  # 검색 창
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    sleep(2)

    # 검색된 정보가 있는 경우에만 탐색
    # 1번 페이지 place list 읽기
    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')
    place_lists = soup.select('.placelist > .PlaceItem') # 검색된 장소 목록

    # 검색된 첫 페이지 장소 목록 크롤링하기
    crawling(place, place_lists)
    search_area.clear()

    # 우선 더보기 클릭해서 2페이지
    try:
        driver.find_element_by_xpath('//*[@id="info.search.place.more"]').send_keys(Keys.ENTER)
        sleep(2)

        # 2~ 5페이지 읽기
        for i in range(2, 6):
            # 페이지 넘기기
            xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
            driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER)
            sleep(2)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            place_lists = soup.select('.placelist > .PlaceItem') # 장소 목록 list

            crawling(place, place_lists)

    except ElementNotInteractableException:
        print('not found')
    finally:
        search_area.clear()


def crawling(place, place_lists):
    """
    페이지 목록을 받아서 크롤링 하는 함수
    :param place: 리뷰 정보 찾을 장소이름
    """

    while_flag = False
    for i, place in enumerate(place_lists):
        # 광고에 따라서 index 조정해야함
        #if i >= 3:
         #   i += 1

        place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
        place_address = place.select('.info_item > .addr > p')[0].text  # place address

        detail_page_xpath = '//*[@id="info.search.place.list"]/li[' + str(i + 1) + ']/div[4]/span[1]/a'
        try:
          driver.find_element_by_xpath(detail_page_xpath).send_keys(Keys.ENTER)
          driver.switch_to.window(driver.window_handles[-1])  # 리뷰 탭으로 변환
          sleep(2)
        
        except:
          pass

        print('####', place_name," 시작")

        # 첫 페이지
        extract_review(place_name)

        # 2-5 페이지
        idx = 3
        try:
            page_num = len(driver.find_elements_by_class_name('link_page')) # 페이지 수 찾기
            for i in range(page_num-1):
                # css selector를 이용해 페이지 버튼 누르기
                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                sleep(2)
                extract_review(place_name)
                idx += 1
            driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 5페이지가 넘는 경우 다음 버튼 누르기
            sleep(2)
            extract_review(place_name) # 리뷰 추출
        except:
            pass

        # 그 이후 페이지
        while True:
            idx = 4
            try:
                page_num = len(driver.find_elements_by_class_name('link_page'))
                for i in range(page_num-1):
                    driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                    sleep(2)
                    extract_review(place_name)
                    idx += 1
                driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 10페이지 이상으로 넘어가기 위한 다음 버튼 클릭
                sleep(2)
                extract_review(place_name) # 리뷰 추출
            except:
                print("no review in crawling")
                break

        driver.close()
        driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환


def extract_review(place_name):
    global driver

    ret = True

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 첫 페이지 리뷰 목록 찾기
    review_lists = soup.select('.list_evaluation > li')

    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        try:
            for i, review in enumerate(review_lists):
              comment = review.select('.txt_comment > span') # 리뷰
              rating = review.select('.grade_star > em') # 별점
              val = ''
              date = driver.find_element_by_xpath('//*[@id="mArticle"]/div[4]/div[4]/ul/li['+str(i+1)+']/div[2]/div/span[3]').text #날짜
              
              if len(comment) != 0:
                  if len(rating) != 0:
                      val1 = comment[0].text
                      val2 = rating[0].text.replace('점', '')
                  else:
                      val1 = comment[0].text + '/0'
                  result = place_name, date, val1, val2
                  result_list.append(result)

        except:
          pass
                        
    else:
        print('no review in extract')
        ret = False

    return ret

def main():
    global driver, load_wb, review_num

    #driver.implicitly_wait(4)  # 렌더링 될때까지 기다린다 4초
    driver.get('https://map.kakao.com/')  # 주소 가져오기

    # 검색할 목록
    place_infos = ['훠궈 맛집']

    for i, place in enumerate(place_infos):
        # delay
        if i % 4 == 0 and i != 0:
            sleep(5)
        search(place)
    result_list.append(place_name)
    result_list.append(place_address)

    driver.quit()
    print("finish")

if __name__ == "__main__":
      main()

#### 파챠이훠궈 광화문점  시작
no review in extract
no review in crawling
#### 완차이면가  시작
no review in crawling
#### 중림 양갈비양꼬치  시작
no review in crawling
#### 훠궈나라 대학로본점  시작
no review in extract
no review in crawling


InvalidSessionIdException: ignored

In [ ]:
len(result_list)

455

In [ ]:
result_list = result_list[:454]

In [ ]:
result_list

[('을지맥옥', '2020.05.31.', '힙지로 분위기좋고 맛도 괜찮음', '3'),
 ('을지맥옥', '2020.05.23.', '가격/분위기/음식 모두 쏘쏘 맥주 먹으러 종종 감', '3'),
 ('을지맥옥', '2020.05.10.', '사진 맛집~!! 움식은 쏘쏘~ 분위기가 지립니당', '5'),
 ('을지맥옥', '2020.03.30.', '예전에 웨이팅 오지게해서 들어갔는데 음 ,, 그저그럼,,, ㅠ 걍 쏘쏘함', '2'),
 ('을지맥옥',
  '2020.03.24.',
  '진짜 입맛이 까다로운편이 아닌데 ,,,,,, 맛에 비해 평점이 너무 높은것같네여 모든 안주가 시더라구요 ,, 재방문의사는 없습니다,,, ',
  '1'),
 ('을지맥옥', '2020.02.22.', '좋아요', '4'),
 ('을지맥옥', '2020.02.18.', '좋아요', '4'),
 ('을지맥옥',
  '2020.01.26.',
  '갠찬. +) 소위 핫플에 자리하고 있는 음식점들과는 다르게 꽤 여러명이 오기에도 적합한 곳 같았다. ',
  '3'),
 ('을지맥옥', '2020.01.25.', '친절. 시끌어수선. 피자는 soso', '3'),
 ('을지맥옥', '2020.01.21.', '무난 시끌 피자맛있음', '4'),
 ('을지맥옥', '2019.10.06.', '분위기좋고 안주도 맛있네용', '5'),
 ('을지맥옥', '2019.08.09.', '별로예요', '1'),
 ('을지맥옥',
  '2019.07.23.',
  '둘이오기엔 양이많고 비쌈.  인테리어와 특유의 감성은 매력적이나   절대 구석에 앉지말것!  소리가울려서 정신이없어서 공황 옴',
  '2'),
 ('을지맥옥', '2019.06.27.', '피자 맛있어용!ㅎㅎㅎㅎ살짝 시끄럽긴 한데 그게 매력인거 같아요ㅎㅎㅎ맥주도 좋았어요', '4'),
 ('을지맥옥',
  '2019.06.20.',
  '피자가 정말 너~어무 맛있음 ! 닭튀김도 바삭하며 짭쪼름하고 살짝 매콤한게 아주 맛남 ',
 

In [ ]:
zhong = str(result_list[454])

In [ ]:
zhong_list = str(zhong.split('),'))

In [ ]:
zhong_list = zhong_list.replace("(","").replace("[","").replace('"',"")

In [ ]:
zhong_split = zhong_list.split(",  ")

In [ ]:
제발 = list(zhong_split)

In [ ]:
zhong_data = pd.DataFrame(제발, columns=['리뷰'])
zhong_data.to_csv('d.csv')

In [ ]:
중식_lists = result

In [ ]:
중식_lists

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pd.DataFrame(result_list, columns=['이름','날짜','리뷰','평점']).to_csv('/content/drive/My Drive/kakaomapreview1') #마라탕.마라샹궈.짬뽕,짜장면

In [ ]:
result_data2 = pd.DataFrame(result_list, columns=['이름','날짜','리뷰','평점'])
result_data2 = result_data2.drop_duplicates(['리뷰'], keep='first')
result_data2.reset_index(drop=False, inplace=False)
result_data2.to_csv('/content/drive/My Drive/중국문화데이터포트폴리오_정혜연/kakaomapreview2') #탕수육, 딤섬, 양꼬치,양갈비

In [ ]:
result_data3 = result_list.append(중식_lists)

In [ ]:
result_data3 = pd.DataFrame(result_list, columns=['이름','날짜','리뷰','평점'])

In [ ]:
result_data3

,이름,날짜,리뷰,평점
0,을지맥옥,2020.05.31.,힙지로 분위기좋고 맛도 괜찮음,3
1,을지맥옥,2020.05.23.,가격/분위기/음식 모두 쏘쏘 맥주 먹으러 종종 감,3
2,을지맥옥,2020.05.10.,사진 맛집~!! 움식은 쏘쏘~ 분위기가 지립니당,5
3,을지맥옥,2020.03.30.,"예전에 웨이팅 오지게해서 들어갔는데 음 ,, 그저그럼,,, ㅠ 걍 쏘쏘함",2
4,을지맥옥,2020.03.24.,"진짜 입맛이 까다로운편이 아닌데 ,,,,,, 맛에 비해 평점이 너무 높은것같네여 모...",1
...,...,...,...,...
449,완차이면가,2018.12.23.,주인공보다 조연인 군만두가 일품. 내용물이 알차고 간이 이미 잘 되어있어 그냥 먹어...,5
450,중림 양갈비양꼬치,2020.09.06.,최고예요,5
451,중림 양갈비양꼬치,2020.08.01.,양꼬치가 맛있습니다. 마파두부와 볶음밥도 시키시길 추천합니다. 마라향이 적절히 나는...,4
452,중림 양갈비양꼬치,2020.07.20.,맛있엉...,5


In [ ]:
result_data3.to_csv('/content/drive/My Drive/중국문화데이터포트폴리오/kakaomapreview3.csv') #중식, 멘보샤, 차이나타운, 중화요리, 홍콩반점, 리춘시장,북경오리,도삭면, 샤오롱바오, 큐큐면관, 탕화쿵푸마라탕, 라화쿵부,라공방,복만루, 건대 마라탕, 훠궈